In [19]:
# install packages as needed
#! pip install adal
#! pip install pandas
#! pip install scikit-learn 



## Utility methods for authenticating and retrieving data from ADLS Gen2 
The next cell contains a series of helper methods which are primarily used to abstract away connectivity, security and enumeration.

All of the secrets will need to be filled in with secrets of your own.

In [20]:
from adal import AuthenticationContext
import requests
import pandas as pd
from datetime import datetime
from io import StringIO, BytesIO

def read_stream_from_adls(endpoint, auth):
    headers = {"Authorization": "Bearer " + auth['accessToken']}
    return requests.get(endpoint, data = None, headers = headers, stream=True)

def read_from_adls(endpoint, auth):
    headers = {"Authorization": "Bearer " + auth['accessToken']}
    return requests.get(endpoint, data = None, headers = headers)

# generate AAD token for REST API authentication
def generate_aad_token():
    resource = "https://storage.azure.com/"
    client_secret = "Lky5pTg2teniHdbeTlmOuivjdIPBPcwMQJ49wGwKiXA="
    client_id = "bd8e91da-c58c-4407-95e0-a8ecad012fc7"
    authority_url = "https://login.microsoftonline.com/72f988bf-86f1-41af-91ab-2d7cd011db47/"
    auth_context = AuthenticationContext(authority_url, api_version = None)
    return auth_context.acquire_token_with_client_credentials(resource, client_id, client_secret)


def type_conveter(input_type):
    switcher = {
        'boolean': 'bool',
        'int64': 'int64'
    }
    return switcher.get(input_type, 'str')

def read_from_adls_with_cdm_format(entity, schema = "cdm"):
    auth = generate_aad_token()
    csv_path = entity.partitions[0].location
    csv_bytes = read_stream_from_adls(endpoint = csv_path, auth = auth).content
    
    # read to pandas dataframe with defined schema from model.json
    names = [attribute.name for attribute in entity.attributes]
    types = dict([(attribute.name, type_conveter(attribute.dataType.value)) for attribute in entity.attributes]) if schema is "cdm" else dict([(attribute.name, 'str') for attribute in entity.attributes])
    
    # Generate the data frame forcing the column names and types to be those from the model.json schema
    buff = BytesIO(csv_bytes)
    df = pd.read_csv(buff, names=names, dtype=types, na_filter = False)
    buff.close()
    return df

## Retrieve CDM specific metadata
The first step is to read a model file that contains information about the CDM Entities that can be used later. This information will be supplied to the helper methods above so that the information in the model file can be used to ensure that the dataframe that is used for modelling at the end of this notebook is correct and matches the model specification.

NOTE: the CdmModel.py file must be available so that it can be imported, the easiest way to do this is simply to have it in the same directory as the notebook, although it can also be referenced as a library. For this notebook the assumption is that its in the same notebook.

In [21]:
# read model.json
import CdmModel

model_endpoint = "https://cdsabyosadev01dxt.dfs.core.windows.net/powerbi/WWI-BG1/WWI-Sales/model.json"
aad_token = generate_aad_token()
model_json = read_from_adls(endpoint = model_endpoint, auth = aad_token).json()
model = CdmModel.Model.fromJson(model_json)

# Scenario: Customer Order Classification

Our hypothesis is that larger customers (by category) will have alrger purchuses (by invoice). Currently we have the following customer category {'Novelty Shop', 'Supermarket', 'Computer Store', 'General Retailer', 'Agent', 'Gift Store', 'Wholesaler', 'Corporate'}

## Preprocessing Data
Before we start modelling we need to do some basic data preparation of the data. 

Our first step is to read the data using the CDM information we got from the model file to enforce column naming and column type for each of the entities.

Once we have good clean dataframes for each of the entities we join them to generate a single flat data frame that is the preferred input for most types of models.

NOTE: The cell below can take some time to execute.

In [22]:
import numpy as np
from IPython.display import display

sales_customer_categories_df = read_from_adls_with_cdm_format(model.entities["Sales CustomerCategories"], "cdm")[['CustomerCategoryID', 'CustomerCategoryName']]
sales_customer_df = read_from_adls_with_cdm_format(model.entities["Sales Customers"], "default")[['CustomerID', 'CustomerCategoryID']]
sales_customer_df = sales_customer_df[['CustomerID', 'CustomerCategoryID']].astype(np.int64)

sales_invoice_line_df = read_from_adls_with_cdm_format(model.entities["Sales InvoiceLines"], "cdm")
sales_invoice_df = read_from_adls_with_cdm_format(model.entities["Sales Invoices"], "cdm")

#Join the 2 elements of invoice together
order_invoice_df = pd.merge(sales_invoice_line_df, sales_invoice_df, on=['InvoiceID'])

#Join customers to their invoices and fix up the datatypes
combined_df = pd.merge(order_invoice_df, sales_customer_df, on=['CustomerID'])
combined_df = combined_df[['InvoiceLineID', 'InvoiceID', 'StockItemID', 'PackageTypeID', 'Quantity', 'UnitPrice', 'LineProfit', 'ExtendedPrice', 'DeliveryMethodID', 'CustomerID', 'CustomerCategoryID']]

#These columns come back as object we need them to be floats.
for col in ['UnitPrice', 'LineProfit', 'ExtendedPrice']:
    combined_df[col] = combined_df[col].astype(np.float64)
 
combined_df.head(5)

,InvoiceLineID,InvoiceID,StockItemID,PackageTypeID,Quantity,UnitPrice,LineProfit,ExtendedPrice,DeliveryMethodID,CustomerID,CustomerCategoryID
0,1,1,67,7,10,230.0,850.0,2645.00,3,832,4
1,97,45,164,7,50,112.0,2650.0,6440.00,3,832,4
2,1363,496,8,9,3,240.0,454.5,828.00,3,832,4
3,1364,496,196,7,72,4.1,151.2,339.48,3,832,4
4,4050,1296,64,7,9,30.0,135.0,310.50,3,832,4


## Create Models
Using sklearn we will take the CDM data and build a (simple) machine learning model from it. In this case we are going to build a simple logistic regression model just to demonstrate the process and approach.

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#Set up the target and feature columns before splitting into training and testing
target_df = combined_df['CustomerCategoryID']
features_df = combined_df.drop(['CustomerCategoryID'], axis = 1) 
X_train, X_test, y_train, y_test = train_test_split(features_df, target_df, test_size = 0.3)

lr = LogisticRegression()

lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

lr_accuracy = lr.score(X_test, y_test)
prob = lr.predict_proba(X_test)[:,1]

print("Logistic Regression: " + str(round(lr_accuracy,2)))

Logistic Regression: 0.71


The accuracy of the result should be 0.71, which is not especially good. In a real modelling exercise we would go back and tweak the columns in the model, featurise them and also potentially experiment with parameters to the LR model. All in the hope that a more accurate model is possible.